In [ ]:
!pip install numpy
!pip install matplotlib
!pip install opencv-python
!pip install scikit-image
!pip install PyWavelets
!pip install PyDrive

In [ ]:
#Let's get the Task2 file

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#https://drive.google.com/file/d/13hOJmMQvd8qEx1uZbQzC_14Uv98omW6z/view?usp=sharing
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '13hOJmMQvd8qEx1uZbQzC_14Uv98omW6z'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('Task2.mat')  
# print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
import scipy.io
mat = scipy.io.loadmat('Task2.mat')

In [ ]:
mat['A'].shape

(5, 5, 256)

In [ ]:
mat['Image'].shape

(5, 5)

In [ ]:
#Import the libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.measure import compare_ssim as ssim
import pywt

In [ ]:
A = mat['A']
y = mat['Image']


In [ ]:
#All the functions together for this q
A_i = A.shape[0]
A_j = A.shape[1]
x_i = x.shape[0]
x_j = 1

#Step 1
def find_1st_best_atom(A,y,x=x ):
  n = A.shape[1]
  l2norm=0
  min_err = 10000
  for i in range(n):
    l2norm = np.linalg.norm(np.dot(A[:,i].reshape((A_i,1)),x[i,:].reshape((1,x_j)))-y,2)
    if(l2norm<min_err):
      min_err= l2norm
      min_err_1st_i= i
    
  return i,min_err

#we must keep track of the best atom indices 
known_best_atoms_i = []

#Step 2
#find other atoms that will minimise the error 
def find_other_atoms(A,y,min_err,x=x ):
  n = A.shape[1]
  min_err_other_i = -1
  for i in range(n):
    if(i not in known_best_atoms_i):
      l2norm = np.linalg.norm(np.dot(A[:,i].reshape((A_i,1)),x[i,:].reshape((1,x_j)))-y,2)
      if(l2norm+min_err<min_err):
        min_err = l2norm + min_err
        min_err_other_i= i
    
  return min_err_other_i,min_err

#Limit of max number of non zero values in x
S_max = 2
i1,min_err = find_1st_best_atom(A,y)
known_best_atoms_i.append(i1)
print(known_best_atoms_i)
for j in range(S_max-1):
  i_other,min_err = find_other_atoms(A,y,min_err)
  if(i_other!=-1):
    known_best_atoms_i.append(i_other)
  print(known_best_atoms_i)

#Make other elements in x whose row numbers are not in 
#known_best_atoms as 0 
def make_sparse_x(x,nonzero_inices = known_best_atoms_i):
  x_rows = x.shape[0]
  for i in range(x_rows):
    if(i not in nonzero_inices):
      x[i,0]=0
  return x

[63]
[63]


In [ ]:
sparse = make_sparse_x(x)
sparse